In [1]:
import pandas as pd
import os
import glob
import underthesea as uts
import numpy as np

In [2]:
path = os.getcwd()
datapath = glob.glob(os.path.join(path,"data/*.txt"))
fulldata = []
for i in datapath:
    with open(i,'r',encoding='utf-8') as f:
#     lines = [i.strip() for i in f.readlines()]
        lines = f.readlines()
    for j in lines:
        fulldata.append(j)

print(fulldata[:5])
print(len(fulldata))

['Thích đánh lộn không? __eou__ Ngon nhà vô __eou__\n', 'Solo yasua không __eou__ Chấp lun 2 mạng đầu __eou__\n', 'Mai đi picnic không? __eou__ Mai bận học rồi __eou__\n', 'Mai học ca mấy vậy? __eou__ Mai học ca 3 __eou__\n', 'Còn tiền không? __eou__ Còn chết liền __eou__\n']
5855


In [3]:
data = [i.split('__eou__') for i in fulldata]
print(data[:5])
data_question = [i[0] for i in data]
data_answer = [i[1] for i in data]
print(data_question[:5])

[['Thích đánh lộn không? ', ' Ngon nhà vô ', '\n'], ['Solo yasua không ', ' Chấp lun 2 mạng đầu ', '\n'], ['Mai đi picnic không? ', ' Mai bận học rồi ', '\n'], ['Mai học ca mấy vậy? ', ' Mai học ca 3 ', '\n'], ['Còn tiền không? ', ' Còn chết liền ', '\n']]
['Thích đánh lộn không? ', 'Solo yasua không ', 'Mai đi picnic không? ', 'Mai học ca mấy vậy? ', 'Còn tiền không? ']


In [4]:
data_table = pd.DataFrame(list(zip(data_question,data_answer)),columns={'question','answer'})
print(data_table)

                                question                 answer
0                 Thích đánh lộn không?            Ngon nhà vô 
1                      Solo yasua không    Chấp lun 2 mạng đầu 
2                  Mai đi picnic không?        Mai bận học rồi 
3                   Mai học ca mấy vậy?           Mai học ca 3 
4                       Còn tiền không?          Còn chết liền 
...                                  ...                    ...
5850                     nhà bạn ở đâu?             Ở Cao lãnh 
5851  cho mình hỏi bạn sống ở quận nào?       Tui ở quận 8 ông 
5852               hiện bạn đang ở đâu?        Tp. Hồ Chí Minh 
5853      địa chỉ cụ thể nhà bạn là gì?     143 Nguyễn Thị Tần 
5854               nhà bạn ở đường nào?      Mới nói ở trên đó 

[5855 rows x 2 columns]


In [5]:
stopwords_list = ['bị','bởi','cả','các','cái','cần','càng','chỉ','chiếc','cho','chứ','chưa','có','có_thể','cứ','cùng','cũng','đã','đang','để','do','đó','được','gì','khi','không','là','lại','lên','lúc','mà','mỗi','này','nên','nếu','ngay','nhiều','như','nhưng','những','nơi','nữa','phải','qua','ra','rằng','rất','rồi','sau','sẽ','theo','thì','từ','từng','và','vẫn','vào','vậy','vì','việc','với']

In [6]:
def my_tokenizer(document):
    words = uts.word_tokenize(document)
    
    pos_tags = uts.pos_tag(words)
    
#     non_stopwords = [w for w in pos_tags if not w[0].lower() in stopwords_list]
    
#     non_punctuation = [w for w in non_stopwords if not w[0] in string.punctuation]
    

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

In [8]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=uts.word_tokenize) #Still not use tokenizer
tfidf_matrix = tfidf_vectorizer.fit_transform(data_question)
count_vectorizer = CountVectorizer(tokenizer=uts.word_tokenize) #Still not use tokenizer
count_matrix = count_vectorizer.fit_transform(data_question)

print(tfidf_matrix.shape)
print(count_matrix.shape)

(5855, 2474)
(5855, 2474)


In [9]:
def ask_question(question):
    query_vect = tfidf_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)
    max_similarity = np.argmax(similarity, axis=None)
    return data_answer[max_similarity]

def ask_question_count(question):
    query_vect = count_vectorizer.transform([question])
    similarity = cosine_similarity(query_vect, count_matrix)
    max_similarity = np.argmax(similarity, axis=None)a
    return data_answer[max_similarity]

In [10]:
print("TFIDF Vectorizer: "+ ask_question('Crush bao lâu rồi?'))
print("Count Vectorizer: "+ ask_question_count('Crush bao lâu rồi?'))

TFIDF Vectorizer:  Khoảng 10 năm 
Count Vectorizer:  Khoảng 10 năm 


In [11]:
predict = [ask_question(i) for i in data_table['question']]
print(predict[:10])
predict_count = [ask_question_count(i) for i in data_table['question']]
print(predict_count[:10])

[' Ngon nhà vô ', ' Chấp lun 2 mạng đầu ', ' Mai bận học rồi ', ' Mai học ca 3 ', ' Còn chết liền ', ' 11:00 rồi ', ' 1 tháng rồi ', ' Mình có rồi ', ' Khoảng 10 năm ', ' Có rồi huhu ']
[' Ngon nhà vô ', ' Chấp lun 2 mạng đầu ', ' Mai bận học rồi ', ' Mai học ca 3 ', ' Còn chết liền ', ' 11:00 rồi ', ' 1 tháng rồi ', ' Mình có rồi ', ' Khoảng 10 năm ', ' Có rồi huhu ']


In [12]:
data_table['predict-tfidf'] = predict
data_table['predict-count'] = predict_count
print(data_table.head())

                 question                 answer          predict-tfidf  \
0  Thích đánh lộn không?            Ngon nhà vô            Ngon nhà vô    
1       Solo yasua không    Chấp lun 2 mạng đầu    Chấp lun 2 mạng đầu    
2   Mai đi picnic không?        Mai bận học rồi        Mai bận học rồi    
3    Mai học ca mấy vậy?           Mai học ca 3           Mai học ca 3    
4        Còn tiền không?          Còn chết liền          Còn chết liền    

           predict-count  
0           Ngon nhà vô   
1   Chấp lun 2 mạng đầu   
2       Mai bận học rồi   
3          Mai học ca 3   
4         Còn chết liền   


In [14]:
print(accuracy_score(data_table['predict-tfidf'],data_table['answer']))
print(accuracy_score(data_table['predict-count'],data_table['answer']))

0.8278394534585825
0.8278394534585825


In [15]:
print(ask_question('còn bao lâu nữa?'))

 dạ còn chị. 
